In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv

from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_regression
from sklearn.datasets import load_boston
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')
np.random.seed(42)

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
ames_df_test = pd.read_csv('../data/ames_iowa_data_test_data_clean', index_col='Id')
X_train_sc = pd.read_csv('../pickle/X_train_sc.csv', header=None)
X_test_sc = pd.read_csv('../pickle/X_test_sc.csv', header=None)
y_train = pd.read_csv('../data/y_train.csv', header=None)
y_test = pd.read_csv('../data/y_test.csv', header=None)
X_train = pd.read_csv('../data/X_train.csv', header=None)
X_test = pd.read_csv('../data/X_test.csv', header=None)

In [3]:
ames_df_test_object = ames_df_test.select_dtypes(exclude=['int64', 'float64'])
ames_df_numeric_test = ames_df_test.select_dtypes(exclude='object')
ames_df_object_dummies_test = pd.get_dummies(ames_df_test_object, drop_first=True)
ames_features_test = pd.concat([ames_df_numeric_test, ames_df_object_dummies_test], axis=1)

In [4]:
ames_df = pd.read_csv('../data/ames_iowa_data_clean', index_col='Id')
ames_df_objects = ames_df.select_dtypes(exclude=['int64', 'float64'])
ames_df_numeric = ames_df.select_dtypes(exclude='object')
ames_df_object_dummies = pd.get_dummies(ames_df_objects, drop_first=True)
ames_features = pd.concat([ames_df_numeric, ames_df_object_dummies], axis=1)
ames_features.drop('saleprice', axis=1, inplace=True)

#### Setup the Pipeline

In [5]:
pipe = Pipeline([
    ('var_thresh', VarianceThreshold()),
    ('ss', StandardScaler()),
    ('kbest', SelectKBest(f_regression)),
    ('enet', ElasticNet())
])

#### GridSearch

In [6]:
params = {
    'var_thresh__threshold': [0, .05],
    'kbest__k': [30, 60],
    'enet__alpha': np.logspace(-3, 3, 7),
    'enet__l1_ratio': [.0001, .3, .5, .7, .9, 1]
}

In [7]:
gs = GridSearchCV(pipe, params)

In [8]:
X_train.shape

(1518, 213)

In [9]:
(X_train.var() < .05).sum()

122

In [10]:
gs.fit(X_train, y_train)

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

...

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/nelson/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('var_thresh', VarianceThreshold(threshold=0.0)), ('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kbest', SelectKBest(k=10, score_func=<function f_regression at 0x1a11284488>)), ('enet', ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'var_thresh__threshold': [0, 0.05], 'kbest__k': [30, 60], 'enet__alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'enet__l1_ratio': [0.0001, 0.3, 0.5, 0.7, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [11]:
gs.score(X_train, y_train)

0.8570266908004481

In [12]:
gs.score(X_test, y_test)

0.8798756509997986

In [13]:
gs.best_params_

{'enet__alpha': 0.1,
 'enet__l1_ratio': 0.0001,
 'kbest__k': 60,
 'var_thresh__threshold': 0}

In [14]:
gs.best_estimator_.named_steps['var_thresh'].get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
gs.best_estimator_.named_steps['kbest'].get_support()

array([False,  True,  True,  True, False,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False,  True, False, False,  True,  True,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,

In [16]:
gs.best_estimator_.named_steps['enet'].coef_

array([ -640.07545553,  4593.62160793, 15373.9227466 ,  1070.71830082,
        5654.85418712,  2974.45642239,  2260.76349364,  2890.84151729,
        3235.30516164,  6221.50765851,  8146.98254972,  3914.84724656,
        3960.18117068,  1743.2798661 ,  2273.58490539,  6290.09552725,
        5046.764059  ,  4025.46805403,  1577.3935266 ,   275.58154219,
        -422.00128193, -2577.96982836,  -789.41744193,  3325.91059627,
        -641.62416449,  -428.17823143,  6051.27409174,  6874.20383633,
        -756.97970863,  5771.49239829, -3684.64307783, -1047.00347005,
        1976.03724539,  -189.39038289,  -227.42773011,  2231.47765715,
        -111.96431681,  1085.25170372,  1691.32830618, -3728.83214856,
       -5905.59538282,  -555.6332276 ,  1902.46077875, -5233.57992301,
       -1319.91098825,  4284.81056454, -2047.19907432,  1816.26955248,
       -2685.19832545,  2226.12713136,    68.28615265, -6788.26759191,
       -8309.33790357,  2889.00757462,   768.75422847,   913.7770423 ,
      

In [17]:
columns = ames_features.columns
columns = columns[gs.best_estimator_.named_steps['var_thresh'].get_support()]
columns = columns[gs.best_estimator_.named_steps['kbest'].get_support()]

pd.DataFrame(gs.best_estimator_.named_steps['enet'].coef_, 
             index = columns, columns=['Coef Weights']).T


lot_frontage     lot_area  overall_qual   year_built  \
Coef Weights   -640.075456  4593.621608  15373.922747  1070.718301   

              year_remod/add  mas_vnr_area  bsmtfin_sf_1  total_bsmt_sf  \
Coef Weights     5654.854187   2974.456422   2260.763494    2890.841517   

               1st_flr_sf   2nd_flr_sf      ...        electrical_SBrkr  \
Coef Weights  3235.305162  6221.507659      ...               68.286153   

              kitchen_qual_Gd  kitchen_qual_TA  garage_type_Attchd  \
Coef Weights     -6788.267592     -8309.337904         2889.007575   

              garage_type_BuiltIn  garage_type_Detchd  garage_qual_TA  \
Coef Weights           768.754228          913.777042      -2838.3696   

              paved_drive_Y  sale_type_New  sale_type_WD   
Coef Weights    3035.059993     2593.65218    -221.754577  

[1 rows x 60 columns]

In [18]:
pred_gs = gs.predict(X_test)

In [19]:
np.sqrt(mean_squared_error(y_test, pred_gs)) ## RMSE

28205.70803433927

In [20]:
pred_gs.shape

(507,)

In [21]:
y_test.shape

(507, 1)

In [22]:
plt.figure(figsize=(12,8))
sns.regplot(pred_gs, y_test[0] , color='red')
plt.title("GridSearch Test Predictions")
plt.xlabel("Predicted Values")
plt.ylabel("Actual Values")

Text(0,0.5,'Actual Values')

#### GridSearch KBest Predictions

In [23]:
diff_cols = set(ames_features.columns).difference(ames_features_test.columns)

In [24]:
for col in diff_cols:
    ames_features_test[col]=0

In [25]:
kaggle = ames_features_test[ames_features.columns]

In [26]:
pred_gs_final = gs.predict(kaggle)

In [27]:
final_gs_df = pd.DataFrame(pred_gs_final, index=kaggle.index, columns=['saleprice'])

In [28]:
final_gs_df.sort_index(inplace=True)

In [29]:
final_gs_df.to_csv('../data/gs_enet_predictions2')